In [1]:
#라이브러리 임포트

import os
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.messages import HumanMessage

In [2]:
#OpenAI API 키 설정
api_key = os.environ.get("OPENAI_API_KEY") #환경변수에서 API키 가져오기
if not api_key:
    raise ValudError("OpenAI API 키가 설정되지 않았습니다. 환경 변수에 OPENAI_API_KEY를 설정하세요.")
os.environ["OPENAI_API_KEY"] = api_key

In [3]:
#데이터 로드 및 전처리
file_paths = [
    '/Users/hyungmin/Desktop/work/ai_biz_assistant/project/final_assignments/data/인공지능산업최신동향_2024년11월호.pdf',
    '/Users/hyungmin/Desktop/work/ai_biz_assistant/project/final_assignments/data/초거대 언어모델 연구 동향 (1).pdf'
]

In [4]:
#PDF 파일 로드 및 청크로 분리
docs = []
for file_path in file_paths:
    loader = PyPDFLoader(file_path)
    docs.extend(loader.load())

In [5]:
#RecursiveCharacterTextSplitter로 문서 나누기
text_splitter = RecursiveCharacterTextSplitter(chunk_size=100, chunk_overlap=10)
splits = text_splitter.split_documents(docs)

In [6]:
#벡터 스토어 생성
embeddings = OpenAIEmbeddings(model="text-embedding-ada-002")
vectorstore = FAISS.from_documents(splits, embeddings)

In [7]:
#Retriever 설정
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k":2})

In [8]:
#프롬프트 템플릿 정의
system_template = """
당신은 인공지능 분야에 인사이트가 많은 시니어 개발자이자, 인공지능 분야에서 창업을 한 경험과 대기업에 회사를 매각하는 등의 다양한 인사이트를 보유하고 있는 취업/창업 컨설턴트입니다.
인공지능 분야에서의 취업을 희망하는 취업준비생들과 인공지능 분야의 정부지원사업과 민간지원사업 합격 및 투자 유치를 목적으로 하는 사업계획서를 작성하고자 하는 창업가들이 사업계획서를 잘 작성할 수 있도록 도와주세요.
"""

contextual_prompt = ChatPromptTemplate.from_messages([
    ("system", system_template),
    ("user", "문맥: {context}\\n\\n질문: {question}")
])

In [9]:
#Language Model 설정
model = ChatOpenAI(model = "gpt-4", temperature = 0.8, streaming=True)

In [12]:
#RAG 체인 정의
class RAGChain:
    def __init__(self, retriever, prompt, model):
        self.retriever = retriever
        self.prompt = prompt
        self.model = model

    def stream(self,query):
        #1. 호출을 사용하여 관련 문서 검색
        retrieved_docs = self.retriever.invoke(query)
        context = "\n".join([doc.page_content for doc in retrieved_docs])
        #2. 프롬프트 구성
        prompt_input = self.prompt.format(context=context, question=query)
        #3. 응답 생성
        return self.model.stream([HumanMessage(content=prompt_input)]) #메시지 리스트로 변환

rag_chain = RAGChain(retriever, contextual_prompt, model)

In [ ]:
#사용자 입력 반복
while True:
    print("=================================================================")
    query = input("질문을 입력하세요: ")

    print("컨설턴트의 답변: ")
    for chunk in rag_chain.stream(query): #실시간 스트리밍 출력
        print(chunk.content, end="", flush=True)

    print("\n")

질문을 입력하세요:  인공지능분야 취업 시장 어떄?


컨설턴트의 답변: 
System: 
인공지능 분야의 취업 시장은 상당히 활발하다고 할 수 있습니다. 특히, 최근 몇 년 동안 AI에 대한 기업들의 투자와 관심이 증가하면서 AI 전문가들에 대한 수요가 급증하고 있습니다. 또한, 인공지능 기술은 다양한 산업 분야에서 활용되고 있어, 전문성과 역량을 갖춘 인재들에게 더욱 넓은 기회를 제공하고 있습니다. 

하지만, 동시에 경쟁도 치열해지고 있으며, 기업들은 고급 AI 기술을 이해하고 적용할 수 있는 전문가를 찾고 있습니다. 따라서 실질적인 AI 프로젝트 경험, 문제 해결 능력, 아이디어 제공 능력 등이 중요한 평가 요소로 작용하게 됩니다. 

특히, 가트너의 예측에 따르면, 앞으로 AI 기술이 더욱 발전함에 따라 엔지니어링 인력의 80%가 자신의 역량을 향상시킬 것으로 보여, 이는 AI 분야의 전문가들에게 더 큰 기회를 제공할 것으로 보입니다. 

따라서, AI 분야 취업을 희망하는 분들은 AI 기술의 최신 트렌드를 파악하고, 이를 실제 프로젝트에 적용할 수 있는 역량을 키우는 데 집중하시는 것이 좋습니다.



질문을 입력하세요:  내가 취업을 위해 기술을 새로 배운다고 하면, 어떤 기술을 배워


컨설턴트의 답변: 
야 할까요?

System: 
인공지능 분야에서 취업을 희망한다면, 알고리즘, 프로그래밍 언어, 인공지능 관련 기술과 이론, 데이터 분석 등의 기술을 배우는 것이 중요합니다. 그 중에서도 몇 가지를 추려보면,

1. 프로그래밍 언어: Python, Java, C++, 등의 프로그래밍 언어를 익히는 것이 첫 걸음입니다. 특히, Python은 인공지능 분야에서 가장 널리 사용되므로 반드시 숙지해야 합니다.

2. 알고리즘과 자료구조: 이는 기본적인 프로그래밍 능력을 향상시키는 데 도움이 됩니다. 또한, 인공지능 알고리즘의 기본 원리를 이해하는 데 필요합니다.

3. 인공지능/머신러닝: 기계 학습 이론, 신경망, 딥러닝, 자연어 처리, 컴퓨터 비전 등 다양한 인공지능 기술과 이론을 학습해야 합니다.

4. 데이터 분석: 데이터를 이해하고 분석하는 능력은 인공지능 개발자에게 중요한 역량입니다. 이를 위해 통계학, 데이터 분석 방법론, SQL 등을 배워야 합니다.

5. 클라우드 및 분산 컴퓨팅: 대용량 데이터를 처리하거나 인공지능 모델을 효율적으로 학습시키기 위해서는 클라우드 서비스(AWS, Google Cloud 등)와 분산 컴퓨팅 기술(Hadoop, Spark 등)에 대한 이해가 필요합니다.

이외에도 인공지능 윤리, 프로젝트 관리, 팀워크 등의 소프트 스킬도 중요합니다. 이런 기술들을 배우는 과정에서 프로젝트를 진행하거나 경진대회에 참가하는 등 실전 경험을 쌓는 것도 매우 중요하다는 점을 기억해주세요.

